## Advanced Lane Finding Project

## Step 1: Calibrate Camera

In [3]:
import numpy as np
import cv2
import glob
import matplotlib
import matplotlib.pyplot as plt
%matplotlib qt

objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

objpoints = []
imgpoints = []

#average vars
retSum = 0.0
mtxSum = np.ndarray((3,3), np.dtype('float64'))
distSum = np.ndarray((1,5), np.dtype('float64'))
rvecsSum = np.array([[0.0], 
                      [0.0],
                      [0.0]], np.dtype('float64'))
tvecsSum = np.array([[0.0], 
                      [0.0],
                      [0.0]], np.dtype('float64'))

mtxDivide = np.ndarray((3,3), np.dtype('float64'))
mtxDivide.fill(20)

distDivide = np.ndarray((1,5), np.dtype('float64'))
distDivide.fill(20)

rvecsDivide = np.array([[0.0], 
                      [0.0],
                      [0.0]], np.dtype('float64'))
rvecsDivide.fill(20)

tvecsDivide = np.array([[0.0], 
                      [0.0],
                      [0.0]], np.dtype('float64'))
tvecsDivide.fill(20)


# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Go through the list and look for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        #ret = float, mtx = mumpy.ndarray, dist = numpy.ndarray, rvecs = list, tvecs = list
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
        retSum += ret
        mtxSum += mtx
        distSum += dist
        rvecsSum += rvecs[0]
        tvecsSum += tvecs[0]      
        cv2.waitKey(500)

retAv = retSum / 20
mtxAv = mtxSum / mtxDivide
distAv = distSum / distDivide
rvecsAv = [rvecsSum[0] / rvecsDivide]
tvecsAv = [tvecsSum[0] / tvecsDivide]

cv2.destroyAllWindows()
print("Done calibrating")

Done calibrating


## Step 2: Convert video to images

Now the calibration values have been found in order to undistort the video, the next step is to conver the video to images in order to then undistort each image.

In [4]:
clip1 = cv2.VideoCapture("project_video.mp4")

#list images are stored in
images = []

#convert project video to images
while True:
    (grabbed, frame) = clip1.read()

    if not grabbed:
        break
    images.append(frame)
print("Sucessfully converted video to images.")

Sucessfully converted video to images.


## Step 3: Define functions for applying undistortions, thresholds

In [5]:
def undistort(img):
    height,  width = img.shape[:2]
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(width,height),1,(width,height))
    
    # undistort
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

    # crop the image
    x,y,width,height = roi
    dst = dst[y:y+height, x:x+width]
    
    return dst

def colorSpace(img):
    #color channels
    s_channel = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)[:,:,2]
    
    v_channel = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)[:,:,2]

    #apply threshold
    s_thresh = (200, 255)
    s_thresh_max = 255
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    v_thresh = (200, 255)
    v_binary = np.zeros_like(v_channel)
    v_binary[(v_channel >= v_thresh[0]) & (v_channel <= v_thresh[1])] = 1
    
    #combine color channels
    color_binary = np.zeros_like(s_binary)
    color_binary[(v_binary == 1)] = 1

    return color_binary

## Step 4: Apply a perspective transform

In [6]:
def warp(img, type):
    img_size = (img.shape[1], img.shape[0])
    
    src = np.float32([[660, 390],
                      [840, 530],
                      [400, 530],
                      [580, 390]])
    
    dst = np.float32([[900, 0],
                      [900, 600],
                      [390, 600],
                      [390, 0]])
    #perspective transform
    if type == 0:
        M = cv2.getPerspectiveTransform(src, dst)
    #reverse perspective transform
    else:
        Minv = cv2.getPerspectiveTransform(dst, src)
        return Minv
    
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    
    return warped

## Step 5: Find the lines and calculate curvature

In [7]:
def find_lines(img):
    #take a histogram of the bottom half of the image
    histogram = np.sum(img[int(img.shape[0]/2):,:], axis=0)
    #create an output image to draw on and  visualize the result
    out_img = np.dstack((img, img, img))*255
    # Find the peak of the left and right halves of the histogram
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    #window vars
    nwindows = 9
    window_height = np.int(img.shape[0]/nwindows)
    
    #x and y positions of all nonzero pixels in the image
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    #current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    #width of the windows +/- margin and minimum number of pixels found to recenter window
    margin = 100
    minpix = 50
    
    #empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    #go through windows
    for window in range(nwindows):
        #identify window boundaries
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        #append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If found > minpix pixels recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    #concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    #extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
    
    #fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    #gnerate x and y values for plotting
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty

def find_curverad(ploty, left_fitx, right_fix):
    #pixels to meters conversion
    ym_per_pix = 30/720
    xm_per_pix = 3.7/700
    
    y_eval = np.max(ploty)
    #new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fix*xm_per_pix, 2)
    #calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    curverad = (left_curverad + right_curverad)/2

    return curverad

## Step 6: Make a function to find lines, curvature of the lines, and the center offset for each frame

In [8]:
def draw_lines(img): 
    #undistort image
    image = undistort(img)
    #perspective transform
    warped = warp(image, 0)
    Minv = warp(image, 1)
    
    #apply gradient and color space thresholds
    imgThresh = colorSpace(warped)
    
    #find lane lines
    left_fitx, right_fitx, ploty = find_lines(imgThresh)

    #plot lines onto the image
   # Create an image to draw the lines on
    warp_zero = np.zeros_like(imgThresh).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(image, 1, newwarp, 0.3, 0)

    #calculate radius of curvature and display it on the image
    curverad = find_curverad(ploty, left_fitx, right_fitx)
    curvetext = "Radius of Curvature (m): " + str(curverad)
    cv2.putText(result, curvetext, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
    
    #calculate the center offset and display it on the image
    xm_per_pix = 3.7/700 # meters per pixel in x dimensio
    centerOffset = centerOffset = ((right_fitx[0] - left_fitx[0]) / 2 - 320) * xm_per_pix
    centertext = "Center offset (m): " + str(centerOffset)
    cv2.putText(result, centertext, (100, 300), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
    
    return result

## Test each function

In [2]:
# #TESTING UNDISTORT FUNCTION
# image = cv2.imread('camera_cal/calibration1.jpg')
# undistorted = undistort(image)
# cv2.imshow('original', image)
# cv2.imshow('undistort', undistorted)

# cv2.waitKey(0)

#TESTING PERSPECTIVE TRANSFORM
# image = images[0]
# undistorted = undistort(image)
# plt.imshow(undistorted)
# warped = warp(undistorted, 0)
# cv2.imshow('original', undistorted)
# cv2.imshow('perspective transform', warped)

#TESTING GRADIENT
# imgThresh = colorSpace(warped)
# cv2.imshow('original', warped)
# plt.imshow(imgThresh)

# cv2.waitKey(0)

#TESTING LINE FUNCTION
# result = draw_lines(image)
# #result, color_warp = draw_lines(image)

# #cv2.imshow('color warp', color_warp)
# cv2.imshow('result', result)

cv2.waitKey(0)


NameError: name 'cv2' is not defined

## Step 7: Get the lines of each frame and convert to video

In [11]:
#write output video
out = cv2.VideoWriter('output.avi', -1, 20.0, (1200,617))

#loop through every frame and find the lane lines
i = 0
for image in images:
    lines = draw_lines(image)
#     print(lines.shape[0])
#     print(lines.shape[1])
#     cv2.imshow('lines',lines)
#     cv2.waitKey(0)
    out.write(lines)
    print("Frame: ", i)
    i += 1

#stop writing to output video
out.release()
print("Finished converting frames into video.")

Frame:  0
Frame:  1
Frame:  2
Frame:  3
Frame:  4
Frame:  5
Frame:  6
Frame:  7
Frame:  8
Frame:  9
Frame:  10
Frame:  11
Frame:  12
Frame:  13
Frame:  14
Frame:  15
Frame:  16
Frame:  17
Frame:  18
Frame:  19
Frame:  20
Frame:  21
Frame:  22
Frame:  23
Frame:  24
Frame:  25
Frame:  26
Frame:  27
Frame:  28
Frame:  29
Frame:  30
Frame:  31
Frame:  32
Frame:  33
Frame:  34
Frame:  35
Frame:  36
Frame:  37
Frame:  38
Frame:  39
Frame:  40
Frame:  41
Frame:  42
Frame:  43
Frame:  44
Frame:  45
Frame:  46
Frame:  47
Frame:  48
Frame:  49
Frame:  50
Frame:  51
Frame:  52
Frame:  53
Frame:  54
Frame:  55
Frame:  56
Frame:  57
Frame:  58
Frame:  59
Frame:  60
Frame:  61
Frame:  62
Frame:  63
Frame:  64
Frame:  65
Frame:  66
Frame:  67
Frame:  68
Frame:  69
Frame:  70
Frame:  71
Frame:  72
Frame:  73
Frame:  74
Frame:  75
Frame:  76
Frame:  77
Frame:  78
Frame:  79
Frame:  80
Frame:  81
Frame:  82
Frame:  83
Frame:  84
Frame:  85
Frame:  86
Frame:  87
Frame:  88
Frame:  89
Frame:  90
Frame:  9

Frame:  693
Frame:  694
Frame:  695
Frame:  696
Frame:  697
Frame:  698
Frame:  699
Frame:  700
Frame:  701
Frame:  702
Frame:  703
Frame:  704
Frame:  705
Frame:  706
Frame:  707
Frame:  708
Frame:  709
Frame:  710
Frame:  711
Frame:  712
Frame:  713
Frame:  714
Frame:  715
Frame:  716
Frame:  717
Frame:  718
Frame:  719
Frame:  720
Frame:  721
Frame:  722
Frame:  723
Frame:  724
Frame:  725
Frame:  726
Frame:  727
Frame:  728
Frame:  729
Frame:  730
Frame:  731
Frame:  732
Frame:  733
Frame:  734
Frame:  735
Frame:  736
Frame:  737
Frame:  738
Frame:  739
Frame:  740
Frame:  741
Frame:  742
Frame:  743
Frame:  744
Frame:  745
Frame:  746
Frame:  747
Frame:  748
Frame:  749
Frame:  750
Frame:  751
Frame:  752
Frame:  753
Frame:  754
Frame:  755
Frame:  756
Frame:  757
Frame:  758
Frame:  759
Frame:  760
Frame:  761
Frame:  762
Frame:  763
Frame:  764
Frame:  765
Frame:  766
Frame:  767
Frame:  768
Frame:  769
Frame:  770
Frame:  771
Frame:  772
Frame:  773
Frame:  774
Frame:  775
Fram